In [ ]:
!pip install transformers==4.37.2 optimum==1.12.0 --quiet
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
!pip install langchain==0.1.9 --quiet
# !pip install chromadb
!pip install sentence_transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20221105 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.5 requires pdfminer.six==20231228, but you have pdfminer-six 20221105 which is incompatible.


In [ ]:
import json

# Opening JSON file
f_train = open('train_TLQA.json')
f_val = open('val_TLQA.json')
f_test = open('test_TLQA.json')

# returns JSON object as a dictionary
train_data = json.load(f_train)
val_data = json.load(f_val)
test_data = json.load(f_test)

In [ ]:
random_question = "List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020"
random_ground_truth = ["Yim Tae-hee (2010)", "Kim Young-joo (2017, 2018, 2019, 2020)"]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class KnnSearch:
    def __init__(self,data=None, num_trees=None,emb_dim=None):
        self.num_trees=num_trees
        self.emb_dim=emb_dim
        self.model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    def get_embeddings_for_data(self, data_ls):
        #model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        embeddings = self.model.encode(data_ls)
        return embeddings

    def get_top_n_neighbours(self, sentence, data_emb, transfer_data, k): #strategy_emb=None, str_qa=None,k):
        sent_emb = self.get_embeddings_for_data(sentence)
        #data_emb = self.get_embeddings_for_data(transfer_questions)
        top_questions = []

        print("new_emb", sent_emb.shape, data_emb.shape)
        text_sims = cosine_similarity(data_emb,[sent_emb]).tolist()
        results_sims = zip(range(len(text_sims)), text_sims)
        sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)

        #print("text_sims",sorted_similarities[:2])
        for idx, item in sorted_similarities[:k]:
                #if item[0] > 0.45:
                    top_questions.append(transfer_data[idx])

        #text_sims = cosine_similarity(strategy_emb,[sent_emb]).tolist()
        #results_sims = zip(range(len(text_sims)), text_sims)
        #sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)
        #print("text_sims",sorted_similarities[:2])
        #for idx, item in sorted_similarities:
        #        top_questions.append(str_qa[idx])
        return top_questions

In [ ]:
knnSearch = KnnSearch()

In [ ]:
train_questions = [item["question"] for item in train_data]
train_questions_embedded = knnSearch.get_embeddings_for_data(train_questions)

In [ ]:
test_questions = [item["question"] for item in test_data]
test_answers = [item["answers"] for item in test_data]

In [ ]:
from transformers import pipeline

# Load the FlanT5-large model
pl = pipeline("text2text-generation", model="google/flan-t5-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(few_shot_examples, question):
  result = ""
  for few_shot_example in few_shot_examples:
    result += f"Question: {few_shot_example['question']}\n"
    result += f"Answer: {'; '.join(few_shot_example['answers'])}\n\n"

  result += f"Question: {question}\n"
  result += "Answer: "
  return result

In [ ]:
def generate_json(questions, generated_answers, ground_truths):
  data = []
  for question, generated, ground_truth in zip(questions, generated_answers, ground_truths):
      entry = {
          "question": question,
          "generated_answer": generated.split("; "),
          "ground_truth": ground_truth
      }
      data.append(entry)

    # Write the list of dictionaries to a JSON file
  with open("output.json", "w") as json_file:
      json.dump(data, json_file, indent=4)

  print("JSON file created successfully!")

In [ ]:
def test_run(questions, ground_truths, k):
  generated_answers = []
  for question in questions:
    most_similar_questions = knnSearch.get_top_n_neighbours(sentence=question,
                                        data_emb=train_questions_embedded,
                                        transfer_data=train_data,
                                        k=k)

    prompt = generate_prompt(most_similar_questions, question)
    output = pl(prompt)
    generated_answers.append(output[0]['generated_text'])

  generate_json(questions, generated_answers, ground_truths)

In [ ]:
generated_answers = test_run(test_questions[30:40], test_answers[30:40], 7)

new_emb (768,) (3212, 768)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


new_emb (768,) (3212, 768)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors


new_emb (768,) (3212, 768)


...

new_emb (768,) (3212, 768)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


new_emb (768,) (3212, 768)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


JSON file created successfully!
